In [ ]:
# TODO: import .raw files and extract 4 channels

In [12]:
import soundfile as sf
data, samplerate = sf.read('/Users/hyh/Desktop/Test Files/Postfiltered/postfiltered_2020-01-08_09_25_03_2.raw', 
                           channels=4, 
                           samplerate=16000,
                           subtype='PCM_16'
                          )

In [13]:
data.shape

(4580352, 4)

In [8]:
samplerate

16000

In [14]:
4580352/16000/60

4.771199999999999